In [49]:
import pandas as pd
import collections
import os
import pprint
import seaborn as sns
import json
import pprint
import numpy as np

In [43]:
network_profiles = ['PROFILE_CASCADE', 'PROFILE_INTRA_CASCADE', 'PROFILE_SPIKE', 'PROFILE_SLOW_JITTERS', 'PROFILE_FAST_JITTERS']
dynamic_profile_json = collections.defaultdict(list)
bola_profile_json = collections.defaultdict(list)
throughput_profile_json = collections.defaultdict(list)
dynamic_directory = './trace/Dynamic'
bola_directory = './trace/BOLA'
throughput_directory = './trace/Throughput'

In [44]:
for filename in os.listdir(dynamic_directory):
    for p in network_profiles:
        if p in filename:
            dynamic_profile_json[p].append(dynamic_directory + '/' + filename)
for filename in os.listdir(bola_directory):
    for p in network_profiles:
        if p in filename:
            bola_profile_json[p].append(bola_directory + '/' + filename)
for filename in os.listdir(throughput_directory):
    for p in network_profiles:
        if p in filename:
            throughput_profile_json[p].append(throughput_directory + '/' + filename)

In [45]:
nqoe_dynamic_profile = collections.defaultdict(list)
nqoe_bola_profile = collections.defaultdict(list)
nqoe_throughput_profile = collections.defaultdict(list)

In [46]:
def compute_nqoe(trace, profile, abrAlgorithm):
    alpha = 1 # Bitrate reward factor (= chunk duration [segment duration [e.g., 1 seconds]])
    beta = 1000000 # Rebuffering penalty factor (= maximum encoding bitrate level [e.g., 1000 Kbps])
    sigma = 200000 # Playback speed penalty factor (= minimum encoding bitrate level [e.g., 200 Kbps])
    mu = 0.02 # Bitrate switch penalty factor (= 0.02 seconds or = 1 second if the bitrate switch is too important)

    currentBitrate = 1000000
    currentQoE_term1 = 0
    currentQoE_term2 = 0

    len_trace = len(trace)
    for i, t in enumerate(trace):
        currentBitrate = t["currentQuality"]["bitrate"] if t["currentQuality"] else currentBitrate
        currentRebufferingTime = t["currentStallDuration"]
        currentLatency = t["currentLatency"]
        gamma = 0.005 if currentLatency <= 1.1 else 0.01 # live latency penalty factor
        if i == len_trace - 1:
            nextBitrate = 0
        else:
            nextBitrate = trace[i+1]["currentQuality"]["bitrate"] if trace[i+1]["currentQuality"] else currentBitrate
        currentQoE_term1 += (alpha*currentBitrate - beta*currentRebufferingTime - gamma*currentLatency)
        currentQoE_term2 += (mu * abs(nextBitrate - currentBitrate))
    
    nQoE = currentQoE_term1 - currentQoE_term2
    if abrAlgorithm == 'dynamic':
        nqoe_dynamic_profile[profile].append(nQoE)
    elif abrAlgorithm == 'bola':
        nqoe_bola_profile[profile].append(nQoE)
    elif abrAlgorithm == 'throughput':
        nqoe_throughput_profile[profile].append(nQoE)

In [47]:
for p in network_profiles:
    for filename in dynamic_profile_json[p]:
        with open(filename) as f:
            trace = json.load(f)
        compute_nqoe(trace, p, 'dynamic')
    for filename in bola_profile_json[p]:
        with open(filename) as f:
            trace = json.load(f)
        compute_nqoe(trace, p, 'bola')
    for filename in throughput_profile_json[p]:
        with open(filename) as f:
            trace = json.load(f)
        compute_nqoe(trace, p, 'throughput')


-18561796005.437405
-19033541005.36053
-22792595016.34713
-19082437005.4645
-20456871005.554356
-20817348040.002117
-17367763005.219048
-19818558005.290413
-17820050005.234375
-16237498015.459671
-17332587005.338028
-17188016005.46612
-20787018005.37737
-18488132005.506557
-19386292005.384556
-19821845005.435608
-16881368005.258018
-20161747005.51733
-20465412005.74961
-17637816005.3455
-17058727005.011414
-18173428005.31942
-20330529005.423172
-19623825004.99876
-17207583005.25411
-16530111005.11734
-19052035005.441364
-16612957012.633179
-20736916005.487667
-19916960044.684948
-18164074005.099545
-24850135027.551815
-24661805020.151543
-21820335025.663296
-25203770024.597862
-21363625026.1815
-23471885025.174603
-23277090026.658627
-22939590025.69653
-24162905024.712814
-22546765022.39221
-24523340024.62096
-24122025025.250732
-24048445023.20585
-22837505025.096825
-22140290025.419895
-23365450023.701027
-22788595024.604187
-23009615023.474194
-23106315025.03264
-23870605023.506744
-

In [51]:
average_qoe_dynamic = 0
average_qoe_bola = 0
average_qoe_throughput = 0

for p in network_profiles:
    for nqoe_trace in nqoe_dynamic_profile[p]:
        average_qoe_dynamic += np.median(nqoe_trace)
    for nqoe_trace in nqoe_bola_profile[p]:
        average_qoe_bola += np.median(nqoe_trace)
    for nqoe_trace in nqoe_throughput_profile[p]:
        average_qoe_throughput += np.median(nqoe_trace)
average_qoe_dynamic /= len(network_profiles)
average_qoe_bola /= len(network_profiles)
average_qoe_throughput /= len(network_profiles)

In [52]:
average_qoe_dynamic, average_qoe_throughput, average_qoe_bola

(-109162841491.18332, -114861941083.16724, -108078314483.97545)